<a href="https://colab.research.google.com/github/siqueirago/Sistema-de-recomenda-o-por-imagem/blob/main/C%C3%B3digo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📓 Notebook – Sistema de Recomendação por Imagens

In [ ]:
# 1. Importar bibliotecas
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.applications import resnet50
from sklearn.neighbors import NearestNeighbors

In [ ]:
# 2. Carregar dataset Fashion MNIST
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.fashion_mnist.load_data()

In [ ]:
# Expandir para 3 canais (RGB)
x_train = np.stack((x_train,)*3, axis=-1)
x_test = np.stack((x_test,)*3, axis=-1)

# Normalizar
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")

print("Formato do dataset:", x_train.shape, y_train.shape)

In [ ]:
# 3. Carregar modelo pré-treinado
base_model = resnet50.ResNet50(weights='imagenet', include_top=False, pooling='avg')

def extract_features(img_array, model=base_model):
    """Extrai embedding de uma imagem"""
    img = tf.image.resize(img_array, (224, 224))
    x = np.expand_dims(img, axis=0).astype("float32")
    x = x.copy()
    x = resnet50.preprocess_input(x)
    features = model.predict(x, verbose=0)
    return features.flatten()

In [ ]:
# 4. Extrair features para um subconjunto (para economizar tempo no Colab)
N = 5000  # número de imagens utilizadas
features_list = []
for i in range(N):
    features_list.append(extract_features(x_train[i]))
features_list = np.array(features_list)

print("Shape dos vetores de características:", features_list.shape)

In [ ]:
# 5. Criar índice de vizinhos
nn = NearestNeighbors(n_neighbors=5, metric='cosine')
nn.fit(features_list)

In [ ]:
## Função de Recomendação

def recommend(query_idx):
    """Mostra recomendações dado o índice de uma imagem"""
    query_img = x_train[query_idx]
    query_feat = extract_features(query_img).reshape(1, -1)
    distances, indices = nn.kneighbors(query_feat)

    plt.figure(figsize=(12, 6))
    plt.subplot(1, 6, 1)
    plt.imshow(query_img.squeeze(), cmap="gray")
    plt.title("Consulta")
    plt.axis("off")

    for i, idx in enumerate(indices[0]):
        plt.subplot(1, 6, i+2)
        plt.imshow(x_train[idx].squeeze(), cmap="gray")
        plt.title(f"Sim {i+1}")
        plt.axis("off")

    plt.show()

In [ ]:
# Testar recomendação com a imagem de índice 10
recommend(123)